In [1]:
import os # for joining path
import torch

import albumentations
import pretrainedmodels
from apex import amp 
import numpy as np
import pandas as pd
from sklearn import metrics

import torch.nn as nn
from torch.nn import functional as F

from wtfml.data_loaders.image import ClassificationLoader
from wtfml.utils import EarlyStopping
from wtfml.engine import Engine

In [2]:
# Just for removing ssl certificate error
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
pretrainedmodels.__dict__["se_resnext50_32x4d"]() 

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
     

In [4]:
# SE res-next
# we are not going to use the putput of the model, we are gonna use the feature of the model
# Last layer should be changed to adaptive pooling insted of avg pool

class SEResNext50_32x4d(nn.Module):
    def __init__(self, pretrained = "imagenet"):
        super(SEResNext50_32x4d, self).__init__()
        self.model = pretrainedmodels.__dict__["se_resnext50_32x4d"](pretrained = pretrained)
        # If we go deep in the model, we will find that model.features function goes in the layer 4 and gets it's output which is the not the final output, but instead the feature output that helps to determine at the final layer
        self.out = nn.Linear(2048, 1)
        
    def forward(self, image, targets):
        bs, _, _, _ = image.shape
        x = self.model.features(image) # we cropped the final steps before the average pooling part
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.reshape(bs, -1) # reshape to batch size
        out = self.out(x)
        loss = nn.BCEWithLogitsLoss()(
            out,
            targets.reshape(-1, 1).type_as(out)
        )
        return out, loss      

In [6]:
def train(fold):
    training_data_path = 'S:/Skin Cancer Detection/data/jpeg/train_resize/' 
    model_path ='S:/Skin Cancer Detection/data/'
    df = pd.read_csv("S:/Skin Cancer Detection/data/train_folds.csv") 
    device = "cuda"
    epochs = 2
    train_bs = 8
    valid_bs = 16
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    
    df_train = df[df.kfold != fold].reset_index(drop = True)
    df_valid = df[df.kfold == fold].reset_index(drop = True)
    # as we need to perform normalization, we use albumentation library, which performs augmentation and normalization at the same time
    train_aug = albumentations.Compose(
         [
         albumentations.Normalize(mean, std, max_pixel_value = 255.0, always_apply = True)
         ]
    )

    valid_aug = albumentations.Compose(
         [
         albumentations.Normalize(mean, std, max_pixel_value = 255.0, always_apply = True)
         ]
    )  
    
    train_images = df_train.image_name.values.tolist()
    train_images = [os.path.join(training_data_path, i + ".jpg") for i in train_images]
    train_targets = df_train.target.values
    
    valid_images = df_valid.image_name.values.tolist()
    valid_images = [os.path.join(training_data_path, i + ".jpg") for i in valid_images]
    valid_targets = df_valid.target.values
    
    train_dataset = ClassificationLoader(
        image_paths = train_images,
        targets = train_targets,
        resize = None,
        augmentations = train_aug
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = train_bs,
        shuffle = True,
        num_workers = 4
    )
    
    valid_dataset = ClassificationLoader(
        image_paths = valid_images,
        targets = valid_targets,
        resize = None,
        augmentations = valid_aug
    )
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size = valid_bs,
        shuffle = False, 
        num_workers = 4
    )
    
    model = SEResNext50_32x4d(pretrained = "imagenet")
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience = 3,
        mode = "max" # Cause we are using it on AUC # metrics    
    )
    
    model, optimizer = amp.initialize( # apex is used for mixed precision training without occupying lot of memory # fp16 format
        model, 
        optimizer,
        opt_level = "O1",
        verbosity = 0
    )
    
    es = EarlyStopping( # we will stop when Auc STARTS INCREASING
        patience = 5, 
        mode = "max"
    )
    for epoch in range(epochs):
        training_loss = Engine.train(
            train_loader,
            model,
            optimizer,
            device,
            fp16 = True
        )
        predictions, valid_loss = Engine.evaluate(
            train_loader,
            model,
            optimizer,
            device
        )
        predictions = np.vstack((predictions)).ravel()
        auc = metrics.roc_auc.score(valid_targets, predictions) # cause for this step, we kept the shuffle as false in valid data loader
        scheduler.step(auc)
        print(f"epoch={epoch}, auc={auc}")
        es(auc, model, os.path.join(model_path, f"model{fold}.bin"))
        if es.early_stop:
            print("Early Stopping")
            break


In [7]:
if __name__ == "__main__" :
    train(fold = 0)
#     predict(fold = 0)

100%|████████████████████████████████████████████████████████████████| 3727/3727 [1:46:12<00:00,  1.71s/it, loss=0.081]


TypeError: to() received an invalid combination of arguments - got (Adam), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [ ]:
torch.cuda.empty_cache() 

In [ ]:
def predict(fold):
    test_data_path = 'S:/Skin Cancer Detection/data/jpeg/train_resize/' 
    model_path =  'S:/Skin Cancer Detection/data/'
    
    df_test = pd.read_csv('test224/')
    df_test.loc[:, "target"] = 0  # we don't need target it's a fake target
    
    device = "cuda"
    epochs = 50
    test_bs = 16
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)

    # as we need to perform normalization, we use albumentation library, which performs augmentation and normalization at the same time


    test_aug = albumentations.Compose(
         [
         albumentations.Normalize(mean, std, max_pixel_value = 255.0, always_apply = True)
         ]
    )  
    
    test_images = df.test_image_name.values.tolist()
    test_images = [os.path.join(test_data_path, i + ".jpg") for i in test_images]
    test_targets = df_test.target.values

    test_dataset = ClassificationLoader(
        image_paths = test_images,
        targets = test_targets,
        resize = None,
        augmentations = test_aug
    )
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size = test_bs,
        shuffle = False, 
        num_workers = 4
    )
    
    model = SEResNext50_32x4d(pretrained = "imagenet")
    model.load_state_dict(torch.load(os.path.join(model_path, f"model{fold}.bin")))
    model.to(device)
    
    predictions = Engine.predict(
        test_loader,
        model,
        device
    )
    
    return np.vstack((predictions)).ravel()